# 10x scRNA-seq analysis pipeline module 1

__In this notebook we will import the raw count matrix ([Per-molecule read information](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/overview) ) and do some basic inspection of read distributions as well as indentify empty droplets and doublets__ 

__INPUT__  
>>raw count matrix (i.e. H5 file) 


__OUTPUT__
>>[R singlecell data object](https://bioconductor.org/packages/release/bioc/vignettes/SingleCellExperiment/inst/doc/intro.html) conataining raw reads -- after exclusion of empty droplets, doublets, cells with high fraction of mt-RNA, and genes with low total read count  
covariate file inculding fraction mtRNA and the library size (total UMI count) of each cell


In [ ]:
rm(list=ls())
my.libs=c("Seurat","scran","scater","DropletUtils")
lapply(my.libs, require, character.only = TRUE)

In [ ]:
dir=c("~/granuloma_nov2020/input_data/")
out=c("~/granuloma_nov2020/individual_QC/")
w.dir=c("~/granuloma_nov2020/")
sample_ID=c("MF245_CD45")
## make a QC-overview file
inputfile=c("input_MF245_Granuloma_Patient_CD45.RData")
metrics=c("input_file","background_FDR","cells", "mitochRNA_thresh","mitochRNA_excl",
          "doublets","cells_post_QC","genes_post_QC","umi_per_gene","genes_per_cell","UMI_per_cell","UMIs_in_top50_in_perc","HVG_thresh_type","HVG_thresh",
         "HVG","PCs","PCs_assocTECH","resolution","cluster","diffGenes_total")
overview=data.frame(metric=rep(NA,length(metrics)),sample=rep(NA,length(metrics)))
colnames(overview)[2]=c(sample_ID)
row.names(overview)=overview$metric=as.character(metrics)
overview["input_file",2]=as.character(inputfile)
####.     ----- we need marker database  --- here we use myeloid_cell_database
marks=read.csv(paste0("~/granuloma_nov2020/myeloid_cell_database_MW.csv"),header=T)
####. -------  we need to convert all gene IDs to standardised NCBI IDs 
file.exists(paste0(w.dir,"Homo_sapiens.gene_info"))             

In [ ]:
# to be done outside of anaconda environment
##dont forget to preload
#export LD_PRELOAD=/home/matthias/anaconda3/lib/libhdf5.so:/home/matthias/anaconda3/lib/libhdf5_hl.so.100
#dat =Seurat::Read10X_h5("MF320_CD45_sig_pos_scRNAseq_filtered.h5", use.names = TRUE, unique.features = TRUE)

load(paste0(dir,inputfile))

### convert to NCBI SYMBOL

In [ ]:
library(limma)
test2=alias2SymbolUsingNCBI(as.character(rownames(dat)),paste0(w.dir,"Homo_sapiens.gene_info"),required.columns = c("GeneID","Symbol"))
test3=as.data.frame(cbind(test2,as.character(rownames(dat))))
colnames(test3)[3]=c("rowN")
test3$new = ifelse(is.na(test3$Symbol), as.character(test3$rowN), test3$Symbol)
rownames(dat)= as.character(test3$new)

### exclude empty droplets:  
[empty Drops method](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1662-y) for distinguishing between background (droplets with no RNA) from droplets containing RNA based on modeling the background of empty drops and compare them to drops with RNA - thus you do not have to make an expert desicion on keeping only cells with at least 1000 genes as in seurat.  
no threshold -- we could give one e.g. at least 3 UMIs to retain  
lower is a stat parameter to model ambient RNA and cannot be directly translated to number of UMIs 

In [ ]:
empty_thresh=c(0.05)
overview["background_FDR",2]=empty_thresh
dat1=DropletUtils::emptyDrops(dat, lower=400, retain=NULL)   ## very slow !!
dat1$FDR[is.na(dat1$FDR)]=1
paste(table(dat1$FDR <empty_thresh)[2],"droplets are below FDR < 0.05 for not being background noise",sep=" ")
overview["cells",2]=table(dat1$FDR <empty_thresh)[2]
dat=dat[,dat1$FDR < empty_thresh]  
rm(dat1) 

In [ ]:
sce = SingleCellExperiment(assays = list(counts = as.matrix(dat)))
is.mito=grepl("MT-",rownames(sce@assays@data$counts))
perc_mt=apply(sce@assays@data$counts,2,function(x) sum(x[is.mito]/sum(x)))
a=c(seq(0,1,by=0.02)[1:length(seq(0,max(perc_mt),by=0.02))-1],max(perc_mt))
hist(perc_mt,breaks=a,freq=T,main=paste("Expression of mitochondrial genes\n sample:",sample_ID,sep=" "), ylab=c("number of cells"),xlab=c("MT expression in %"))
abline(v=median(perc_mt),col="red",lwd=2)
abline(v=quantile(perc_mt,probs = seq(0,1,by=0.25))[2],col="green",lty=2,lwd=2)  # 25%
abline(v=quantile(perc_mt,probs = seq(0,1,by=0.25))[4],col="green",lty=2,lwd=2)  # 75%
abline(v=(median(perc_mt)+3*(mad(perc_mt))),col="blue",lty=2,lwd=2)
legend("right", legend=c("median", "25% and 75% percentile","outlier threshold (3xMAD)"),
       col=c("red", "green","blue"), lty=c(1,2,2),lwd=1.5, cex=0.8)
              
#######.   ------------------------------------
jpeg(paste0(out,sample_ID,"_mtRNA.jpeg"))
hist(perc_mt,breaks=a,freq=T,main=paste("Expression of mitochondrial genes\n sample:",sample_ID,sep=" "), ylab=c("number of cells"),xlab=c("MT expression in %"))
abline(v=median(perc_mt),col="red",lwd=2)
abline(v=quantile(perc_mt,probs = seq(0,1,by=0.25))[2],col="green",lty=2,lwd=2)  # 25%
abline(v=quantile(perc_mt,probs = seq(0,1,by=0.25))[4],col="green",lty=2,lwd=2)  # 75%
abline(v=(median(perc_mt)+3*(mad(perc_mt))),col="blue",lty=2,lwd=2)
legend("right", legend=c("median", "25% and 75% percentile","outlier threshold (3xMAD)"),
       col=c("red", "green","blue"), lty=c(1,2,2),lwd=1.5, cex=0.8)
              
dev.off()             
              
## ------------     check and subset
table(perc_mt < (median(perc_mt)+3*(mad(perc_mt))))
thresh=median(perc_mt)+3*(mad(perc_mt))
thresh=0.15
exclude_mtDNA=table(perc_mt < thresh)[1]            
sce1=sce[,perc_mt < thresh]
overview["mitochRNA_thresh",2]=thresh
overview["mitochRNA_excl",2]=exclude_mtDNA
dim(sce1)

### remove Doublets 

[Doublet cells](https://ltla.github.io/SingleCellThoughts/software/doublet_detection/bycell.html) for each cell are simultated and compared to the actual neighboing cells of the tested cell (this makes a score)
* Force match corrects for RNA content of cells  
* size factors and normalization is computed internally to avoid confusion  
* This is what happens:  
> 1. Simulate thousands of doublets by adding together two randomly chosen single-cell profiles.  
> 2. For each original cell, compute the density of simulated doublets in the surrounding neighbourhood.  
> 3. For each original cell, compute the density of other observed cells in the neighbourhood.  
> 4. Return the ratio between the two densities as a “doublet score” for each cell.  


so let's say we simmulated douplet cells
> now for a particular cell we are looking at 100 neighbors  
> densitiy of doublets for this cell is 30 that makes 30/100   
> density of other cells is 70/100 than makes 70/100  
> makes a score of 0.4 -- log10 of that  
>> as the number of other cells goes down the score goes up  
> process number of neigbors = now 200  
  
__cutoff should be around 4__

In [ ]:
doublet_score=scran::doubletCells(sce1, k = 200,force.match=TRUE)
doub_score=log10(doublet_score+1)
thresh=median(doub_score)+3*mad(doub_score)
thresh=4
hist(doub_score,breaks=25,main="Distribution of Doublet Scores",xlab="log(doublet scores)",ylab=c("number of cells"))
abline(v=thresh,col="blue",lty=2,lwd=2)
legend("topright", legend=c("Doublet exclussion threshold"),
       col=c("blue"), lty=c(2),lwd=1.5, cex=0.8)

exclude_doublet=table(doub_score<thresh)[1]
sce2=sce1[,doub_score<thresh]
paste("in sample QC process we excluded",exclude_mtDNA,"samples due to hight Mitochondrial DNA proportion and",
      exclude_doublet,"potential Doublet cells",sep=" ")
paste("that leaves ", dim(sce2)[2], "cells for further analysis")
overview["doublets",2]=exclude_doublet
#  --------------------------------------------------------    remove mitochndrial RNA assays 
#is.mito=grepl("MT-",rownames(sce2@assays@data$counts))
#perc_mt1=apply(sce2@assays@data$counts,2,function(x) sum(x[is.mito]/sum(x)))
#paste("we excluded", table(grepl("MT-",rownames(sce2@assays@data$counts)))[2],"mitochondiral genes",sep=" ")
#sce2=sce2[!(grepl("MT-",rownames(sce2@assays@data$counts))),]
## --------------------------------------------------------------

### UMI per cell (library size)

In [ ]:
umi_per_cell=apply(sce@assays@data$counts,2,sum)
median(umi_per_cell)

In [ ]:
umi_per_cell=apply(sce@assays@data$counts,2,sum)
lib_size_mad=mad(umi_per_cell)
a=c(seq(0,max(umi_per_cell/1000),by=3)[1:length(seq(0,max(umi_per_cell/1000),by=3))-1],max(umi_per_cell/1000))
options(warn=-1)
hist(umi_per_cell/1000,freq=T,
     main=paste("UMI per cell (library size)\n sample:",sample_ID,sep=" "), ylab=c("number of cells"),xlab=c("UMI per cell in thousands"))
abline(v=median(umi_per_cell/1000),col="red",lwd=2)
abline(v=quantile(umi_per_cell/1000,probs = seq(0,1,by=0.25))[2],col="green",lty=2,lwd=2)  # 25%
abline(v=quantile(umi_per_cell/1000,probs = seq(0,1,by=0.25))[4],col="green",lty=2,lwd=2)  # 75%
#abline(v=(median(umi_per_cell)+5*(mad(umi_per_cell)))/1000,col="blue",lty=2,lwd=2)
legend("right", legend=c(paste0("median = ",median(umi_per_cell/1000)), "25% and 75% percentile"),
       col=c("red", "green"), lty=1:2,lwd=1.5, cex=0.8, title="width of each bin = 3",title.adj=0)

jpeg(paste0(out,sample_ID,"_LIBRARY_SIZE.jpeg"))
hist(umi_per_cell/1000,breaks=a,freq=T,
     main=paste("UMI per cell (library size)\n sample:",sample_ID,sep=" "), ylab=c("number of cells"),xlab=c("UMI per cell in thousands"))
abline(v=median(umi_per_cell/1000),col="red",lwd=2)
abline(v=quantile(umi_per_cell/1000,probs = seq(0,1,by=0.25))[2],col="green",lty=2,lwd=2)  # 25%
abline(v=quantile(umi_per_cell/1000,probs = seq(0,1,by=0.25))[4],col="green",lty=2,lwd=2)  # 75%
#abline(v=(median(umi_per_cell)+5*(mad(umi_per_cell)))/1000,col="blue",lty=2,lwd=2)
legend("right", legend=c(paste0("median = ",median(umi_per_cell/1000)), "25% and 75% percentile"),
       col=c("red", "green"), lty=1:2,lwd=1.5, cex=0.8, title="width of each bin = 3",title.adj=0)

dev.off()




### Genes per cell

In [ ]:
feature_per_cell=apply(sce@assays@data$counts,2,function(x)table(x %in% c(0))[1])

a=c(seq(0,max(feature_per_cell),by=200)[1:length(seq(0,max(feature_per_cell),by=200))-1],max(feature_per_cell))
hist(feature_per_cell,breaks=a,freq=T,
     main=paste("Feature per cell\n sample:",sample_ID,sep=" "), ylab=c("number of cells"),xlab=c("features per cell"))
abline(v=median(feature_per_cell),col="red",lwd=2)
abline(v=quantile(feature_per_cell,probs = seq(0,1,by=0.25))[2],col="green",lty=2,lwd=2)  # 25%
abline(v=quantile(feature_per_cell,probs = seq(0,1,by=0.25))[4],col="green",lty=2,lwd=2)  # 75%
legend("right", legend=c("median", "25% and 75% percentile"),
       col=c("red", "green"), lty=1:2,lwd=1.5, cex=0.8, title="width of each bin = 200",title.adj=0)

## saturation plot  
>should get flat at the right end  
>if it looks linear --  coverage is too low

In [ ]:
overview["UMI_per_cell",2]=mean(umi_per_cell)
overview["genes_per_cell",2]=mean(feature_per_cell)
sat=as.data.frame(cbind(feature_per_cell,umi_per_cell))
sat=sat[order(sat$feature_per_cell),]
colnames(sat)=c("GENES","nUMI")
p=ggplot(sat,aes(x=nUMI,y=GENES)) +
geom_point(alpha=0.1)+
stat_smooth(method=loess)
p
jpeg(paste0(out,sample_ID,"_saturation.jpeg"))
print(p)
dev.off()

> #### UMI per feature

In [ ]:
umi_per_feature=apply(sce2@assays@data$counts,1,sum)
ave_per_feature=apply(sce2@assays@data$counts,1,mean)
total=dim(sce2)[2]
one=format(mean(umi_per_feature[log10(ave_per_feature)<log10(0.2)]),digits=1)
two=format(mean(umi_per_feature[log10(ave_per_feature)<log10(0.05)]),digits=1)
drei=format(mean(umi_per_feature[log10(ave_per_feature)<log10(0.01)]),digits=1)
four=format(mean(umi_per_feature[log10(ave_per_feature)<log10(0.005)]),digits=1)

hist(log10(ave_per_feature),breaks=50,main=c("Average expression per feature"),xlab=c("log10(mean expression per feature)"),
     ylab=c("number of features"))
abline(v=log10(0.2),col="blue",lty=2,lwd=2)
abline(v=log10(0.05),col="green",lty=2,lwd=2)
abline(v=log10(0.01),col="red",lty=2,lwd=2)
abline(v=log10(0.005),col="black",lty=2,lwd=2)

legend("topright", legend=c(paste("equals",four,"reads distibuted over",total,"cells"), 
                            paste("equals",drei,"reads distibuted over",total,"cells"),
                            paste("equals",two,"reads distibuted over",total,"cells"),
                            paste("equals",one,"reads distibuted over",total,"cells")),
       col=c("black", "red","green","blue"), lty=c(2),lwd=1.5, cex=0.8, title="expression in numbers",title.adj=0)

jpeg(paste0(out,sample_ID,"_UMI_per_feature.jpeg"))
hist(log10(ave_per_feature),breaks=50,main=c("Average expression per feature"),xlab=c("log10(mean expression per feature)"),
     ylab=c("number of features"))
abline(v=log10(0.2),col="blue",lty=2,lwd=2)
abline(v=log10(0.05),col="green",lty=2,lwd=2)
abline(v=log10(0.01),col="red",lty=2,lwd=2)
abline(v=log10(0.005),col="black",lty=2,lwd=2)

legend("topright", legend=c(paste("equals",four,"reads distibuted over",total,"cells"), 
                            paste("equals",drei,"reads distibuted over",total,"cells"),
                            paste("equals",two,"reads distibuted over",total,"cells"),
                            paste("equals",one,"reads distibuted over",total,"cells")),
       col=c("black", "red","green","blue"), lty=c(2),lwd=1.5, cex=0.8, title="expression in numbers",title.adj=0)

dev.off()

In [ ]:
thresh1=c(0.01)
paste("applying a threshold of",thresh1, "will remove",table(log10(ave_per_feature)>log10(thresh1))[1], "genes",sep=" ")
paste(" and leave",table(log10(ave_per_feature)>log10(thresh1))[2],"in our analysis",sep=" ")
overview["genes_post_QC",2]=table(log10(ave_per_feature)>log10(thresh1))[2]
sce3=sce2[log10(ave_per_feature)>log10(thresh1),] 

In [ ]:
doublet_score=scran::doubletCells(sce3, k = 200,force.match=TRUE)
doub_score=log10(doublet_score+1)
mat=sce3@assays@data$counts
dat=CreateSeuratObject(mat,assay = "RNA")
dat@meta.data$doub_score=doub_score
dat[["percent.mt"]] <- PercentageFeatureSet(dat, pattern = "^MT-")
overview["cells_post_QC",2]=dim(dat)[2]
write.csv(overview,file=paste0(out,sample_ID,".csv"))

In [ ]:
head(dat@meta.data)

## SEURAT SCT standard integration and clustering

# restrict to a minimum of 1000 UMIs

In [ ]:
dim(dat)
dat=dat[,dat@meta.data$nCount_RNA > 1000]
dim(dat)

In [ ]:
dat1=SCTransform(dat, variable.features.n = NULL,conserve.memory = F, return.only.var.genes = F)
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
dat1 =CellCycleScoring(dat1, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
dat1$CC.Difference = dat1$S.Score - dat1$G2M.Score



In [ ]:
library(tidyverse)
dat1=SCTransform(dat1, variable.features.n = NULL,vars.to.regress=c("nCount_RNA","percent.mt","CC.Difference"),
            conserve.memory = F, return.only.var.genes = F) %>% RunPCA(verbose = TRUE)

In [ ]:
Seurat::DimHeatmap(dat1, dims =1 , cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::DimHeatmap(dat1, dims = 2, cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::ElbowPlot(dat1)

In [ ]:
sig_PCs=c(1:14)
dat1 <- RunUMAP(dat1,dims = sig_PCs)
sce.seurat1=Seurat::FindNeighbors(dat1,reduction = "pca",k.param = 20,dims=sig_PCs) 

In [ ]:
Idents(sce.seurat1)=as.factor(sce.seurat1@meta.data$Phase)
jpeg(paste0(out,sample_ID,"_cell_cycle.jpeg"))
UMAPPlot(sce.seurat1)
dev.off()
UMAPPlot(sce.seurat1)

In [ ]:
i=c("MMP9")  ## MMP9 MKI67 CD4 CD14 SLC9B2 CD68
#MKI67 SLC9B2 SIGLEC15  COL1A1  KRT5  KRT14 PECAM1  CD14 CD4  PTPRC   CD3E   MRC1  NKG7
p=FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
    print(p)
    #jpeg(paste(out,sample_ID,"_umap_", i,".jpeg",sep=""))
     # print(p)
    #dev.off()

In [ ]:
clust_test=function(sce) {
  cont=as.data.frame(matrix(NA,nrow=length(unique(Idents(sce))), ncol=4))
  colnames(cont)=c("cluster","cells","mean_total_UMI","mean_features")
  i=1
  for (k in unique(Idents(sce))) {
    print(c(k))
    cont[i,1]=c(k)
    dat23=sce@assays$RNA@counts[,Idents(sce)%in%c(k)]
    cont[i,2]=table(Idents(sce)%in%c(k))[2]
    cont[i,3]=mean(apply(dat23,2,function(x)sum(x)))
    cont[i,4]=mean(apply(dat23,2,function(x)table(x %in% c(0))[1]))
    i=i+1
  }
  cont=cont[order(as.numeric(as.character(cont$cluster))),]
  return(cont)
}

In [ ]:
library(ggplot2)
library(ggtree)
library(cowplot)
clust.res=0.3
sce.seurat1=Seurat::FindClusters(sce.seurat1,resolution=clust.res)  
sce.seurat1= Seurat::BuildClusterTree(sce.seurat1,dims=sig_PCs)
#sce.seurat1 <- RunUMAP(sce.seurat1, dims = sig_PCs)
tree1=sce.seurat1@tools$`Seurat::BuildClusterTree`
pl=Seurat::DimPlot(sce.seurat1, reduction = "umap")
pl= pl+ ggtitle(paste("UMAP and phylo-tree visualization of cell clusters \n at resolution",clust.res,sep=" ")) +
  theme (plot.title = element_text(color="black", size=14, face="bold",hjust = 0.5))
pl.matrix=ggplot_build(pl)$data[1][[1]]
uni_pl_mat=pl.matrix[!duplicated(pl.matrix$group),]
tr=ggtree(tree1,layout="circular") + geom_tiplab(aes(angle=angle), lwd=5,offset=.5,fontface="bold")  
plot_grid(pl, tr, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")    
jpeg(paste0(out,sample_ID,"CLUSTERING_OVERVIEW.jpeg"))
 plot_grid(pl, tr, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")
dev.off()
res1=clust_test(sce.seurat1)
res1
#write.csv(res1,file=paste0(out,"CLUSTERING_LEG.csv"))

### testing markers manually

In [ ]:
target=alias2SymbolUsingNCBI("COL1A1",paste0(w.dir,"Homo_sapiens.gene_info"),required.columns = c("GeneID","Symbol"))
i=as.character(target[1,2])


p=FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
    print(p)


### looks like there is a Fibroblast contamination  
> those will be excluded

In [ ]:
head(sce.seurat1@reductions$umap@cell.embeddings[,2])
dim(sce.seurat1)
sce.seurat1=sce.seurat1[,sce.seurat1@reductions$umap@cell.embeddings[,2] > -10]
dim(sce.seurat1)

In [ ]:
marks=read.csv("myeloid_cell_database_MW.csv",header=T)
marks=marks[marks$curator %in% c("MW"),]       ################.  for now --- cut down number a little ....
for (k in (1:nrow(marks))) {
    target=alias2SymbolUsingNCBI(as.character(marks$Name[k]),paste0(w.dir,"Homo_sapiens.gene_info"),required.columns = c("GeneID","Symbol"))
    i=target[1,2]
    if (! i %in% rownames(sce.seurat1)){
        print(noquote(paste(marks$Name[k]," serving as Marker for " ,marks$cell_type[k],"is not in object and will be skipped!")))
        next
    }else {
       p= FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
        p= p + ggtitle(paste(marks$Name[k],"specific for", marks$cell_type[k],"cells?") )
        v=VlnPlot(sce.seurat1, features = as.character(i))    
        print(plot_grid(p, v, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")  )  
    }
    
}

In [ ]:
target=alias2SymbolUsingNCBI("CD3E",paste0(w.dir,"Homo_sapiens.gene_info"),required.columns = c("GeneID","Symbol"))
    i=target[1,2]
target
    if (! i %in% rownames(sce.seurat1)){
        print(noquote(paste(marks$Name[k]," serving as Marker for " ,marks$FeatureType[k],"is not in data object and will be skipped!")))
        next
    }else {
       p= FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
        p= p + ggtitle(paste(marks$Name[k],"specific for", marks$FeatureType[k],"cells?") )
        v=VlnPlot(sce.seurat1, features = as.character(i))    
        print(plot_grid(p, v, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")  )  
    }




## Association testing NOW !!

In [ ]:
sce.seurat=sce.seurat1
clust.n=as.character(unique(Idents(sce.seurat)))
clust.size=table(Idents(sce.seurat))
clust.size

clust.n=as.character(unique(Idents(sce.seurat)))
rm(res_wilcox)



In [ ]:
library(pROC)
library(Matrix)
# ====================================.  prep input data
clust.n=as.character(unique(Idents(sce.seurat)))
clust.size=table(Idents(sce.seurat))
clust.size
max_per_cluster=c(5000)  
clust.n=as.character(unique(Idents(sce.seurat)))
rm(res_wilcox)


### ---------------------------------------------   one vs one for all cluster
for (i in clust.n) {        ##  case loop
     contr.clust=clust.n[!clust.n %in% i]
    for (k in contr.clust) {    ## contr loop
        if (i == k) {
            next
        }  
         print(paste("computing markers for cluster", i,"compared to",k,"now!!!", sep=" "))
          seurat_cluster_0 = FindMarkers(sce.seurat, ident.1 = i, ident.2 = k,test.use = "MAST",
                                 logfc.threshold = 0.15,max.cells.per.ident = max_per_cluster,
                                min.cells.group=50,only.pos = TRUE)  # min.pct = 0.3
          seurat_cluster_0=seurat_cluster_0[seurat_cluster_0$p_val_adj < 0.05,]
        seurat_cluster_0$ID=as.character(rownames(seurat_cluster_0))
          seurat_cluster_0$case_CLUSTER=rep(i,nrow(seurat_cluster_0))
          seurat_cluster_0$contr_CLUSTER=rep(k,nrow(seurat_cluster_0))
          seurat_cluster_0$clust_size=rep(as.numeric(clust.size[i]),nrow(seurat_cluster_0))
          sce.case=sce.seurat[rownames(sce.seurat) %in%  as.character(seurat_cluster_0$ID),Idents(sce.seurat)%in%c(i)]
          sce.contr=sce.seurat[rownames(sce.seurat) %in%  as.character(seurat_cluster_0$ID),Idents(sce.seurat)%in%c(k)]
            seurat_cluster_0=seurat_cluster_0[as.character(rownames(sce.case)), ]
            seurat_cluster_0$case_total_umi=Matrix::rowSums(sce.case@assays$RNA@counts)
            seurat_cluster_0$contr_total_umi=Matrix::rowSums(sce.contr@assays$RNA@counts)
            ##.   ------------- get AUC
            auc.sce=merge(sce.case,sce.contr)
            seurat_cluster_0=seurat_cluster_0[as.character(rownames(auc.sce)), ]
            response=rep(1,length(Idents(auc.sce)))                                       
            response[Idents(auc.sce)%in% c(i)]=2
            seurat_cluster_0$AUC=apply(auc.sce@assays$SCT@data,1,function(x)auc(response,as.numeric(x),quiet=T,allow.invalid.partial.auc.correct=F))             
        if(!exists("res_wilcox")){
            res_wilcox=seurat_cluster_0
          }else {
            res_wilcox=rbind(res_wilcox,seurat_cluster_0)
          }
      rm(seurat_cluster_0,response,add.t,sce.contr,sce.case,auc.sce)
    } ## contr loop
}  ## case loop                                                                  

In [ ]:
save(res_wilcox,file=paste(out,"ASSOC_result_cluster_",sample_ID, ".RData"))

In [ ]:
#save(sce.seurat,file=paste(out,"sce_seurat_",sample_ID,".RData"))

In [ ]:
head(res_wilcox)

In [ ]:
####.   -----------------------   Parameter for enrichemt
library(enrichR)
enrich.lists=c('WikiPathways_2019_Human','KEGG_2019_Human' ,'Reactome_2016','BioPlanet_2019','Human_Gene_Atlas','ARCHS4_Tissues',
               'Mouse_Gene_Atlas','GO_Biological_Process_2018','GO_Molecular_Function_2018','MSigDB_Hallmark_2020')
screen.overlaps=c('MSigDB_Hallmark_2020','Human_Gene_Atlas','ARCHS4_Tissues','Mouse_Gene_Atlas','KEGG_2019_Human')
###      ----------------------   NEW CURRENT VERSION.  !!!!!!!!!
tiering_thresh=data.frame(case_perc=c(0.5,0.3),contr_perc=c(0.05,0.05))
AUC_tier3=c(0.8)
lgFC_tier3=c(0.25)
maxCONTR=length(unique(res_wilcox$contr_CLUSTER))-1
markerN=5
### ----------------------------- tiering function
tiering=function(dat,markerN){
    dat=dat[order(-dat$AUC),]
    order.ID=dat$ID[!duplicated(dat$ID)]   ### returns them in order of apperance !
    ok.ID=order.ID[1:markerN]
    dat12=dat[dat$ID %in% as.character(ok.ID),]
return(dat12)
}
##------------------------------------------
all.cases=unique(res_wilcox$case_CLUSTER)
for (k in all.cases) {
    print(noquote(c('--------------------------------------------------------------------------------')))
    print(noquote(paste0("--------------   processing cluster ",k," as CASES now !!   ------------------------")))
    ## -------------------------------------------------------  prep full MACR dataset
    inter=res_wilcox[res_wilcox$case_CLUSTER %in% k,]
    dim(inter)
    for (t in c("TIER","DUPL","nameCONT","panel","panel_descr",unique(inter$contr_CLUSTER))){
        inter[,t]=NA
    } 
    ### ---------------------------------------------   start
    rm(final.out)
    for (tier in c(1,2,3)){
        if (!exists("final.out")){
            int.t1a=inter[inter$pct.1>tiering_thresh[tier,1] & inter$pct.2 < tiering_thresh[tier,2],]
            int.t1a$TIER=c(tier)
        } else if (tier %in% c(2)) {
            inter1=inter[!(inter$ID %in% final.out$ID),]
            int.t1a=inter1[inter1$pct.1>tiering_thresh[tier,1] & inter1$pct.2 < tiering_thresh[tier,2],]
            int.t1a$TIER=c(tier)
        } else if ( tier %in% c(3)){
            inter1=inter[!(inter$ID %in% final.out$ID),]
            int.t1a=inter1[inter1$AUC> as.numeric(AUC_tier3) & inter1$avg_logFC > lgFC_tier3,]
            int.t1a$TIER=c(tier)
        }
    ### ------------   prep data per TIER
        rm(int.t1)
        for (kt in unique(int.t1a$ID)) {
            int2=int.t1a[int.t1a$ID %in% kt,]
            entr=as.character(int2$contr_CLUSTER)
            int2$DUPL=length(entr)
            int2$nameCONT = paste(entr,sep='',collapse=" | ")
            for (j in entr) {
                int2[,j]=c(1)
                }  
            if(!exists("int.t1")){
                int.t1=int2
                }else {
                int.t1=rbind(int.t1,int2) 
            }
        }
    ### -------------------------------------
    ### create panels -- per TIER   
        spec=NA
        for (r in maxCONTR:1) {  
            dat=int.t1[int.t1$DUPL %in% r,]
            #print (paste("this is control ", r, sep=" "))
            if (max(int.t1$DUPL) < r) {  
                #print (paste("skip ",r ,"!!!"))
                next 
                } else if (any(dat$DUPL %in% maxCONTR)) {
                        #print (c("found a lot of data"))         
                        dat12=tiering(dat,markerN)
                        spec=c(spec,as.character(dat12$contr_CLUSTER))
                        int.t1$panel[int.t1$ID %in% unique(dat12$ID)]=paste0("TIER",tier,"_panel")        
                    } ### a lot of data else if
                    else if (nrow(dat) > 1 & length(spec)<2){
                        #print (c("found data for the first time"))   
                        dat12=tiering(dat,markerN)
                        spec=c(spec,as.character(dat12$contr_CLUSTER))
                        int.t1$panel[int.t1$ID %in% unique(dat12$ID)]=paste0("TIER",tier,"_panel")   
                      }
                    else if (nrow(dat) > 1){
                       # print (c("found data !!!")) 
                        freq.t=as.data.frame(table(spec))
                        stay.in=freq.t$spec[freq.t$Freq < markerN]
                        dat=dat[dat$contr_CLUSTER %in% stay.in, ]
                        if (nrow(dat) > 1){
                            dat12=tiering(dat,markerN)
                            spec=c(spec,as.character(dat12$contr_CLUSTER))
                            int.t1$panel[int.t1$ID %in% unique(dat12$ID)]=paste0("TIER",tier,"_panel") 
                        } else { next }
                    }
        } # for loop
###. -------
        fin.int=as.data.frame(table(int.t1$contr_CLUSTER[!is.na(int.t1$panel)]))
        zeros=unique(inter$contr_CLUSTER)[!(unique(inter$contr_CLUSTER) %in% fin.int$Var1)]
        if (length(zeros >0)){
            fin.int=rbind(fin.int,data.frame(Var1=zeros,Freq=rep(0,length(zeros))))
        }
        pan.descr=paste(paste(fin.int$Freq, fin.int$Var1,sep="x"),sep="",collapse=" ")
        int.t1$panel_descr[int.t1$panel %in% paste0("TIER",tier,"_panel")] = c(pan.descr)  
## 
        print(noquote(paste0(" TIER ",tier, " Markerpanel:")))
        print(noquote(paste(unique(int.t1$ID[int.t1$panel %in% paste0("TIER",tier,"_panel") ]))))
        if(!exists("final.out")){
            final.out=int.t1
        } else {
            final.out=rbind(final.out,int.t1)
        }
    } # close tiering loop
    assign(paste0("all_tiers_", k), final.out)  ### keep this for phenotype loop
    write.table(final.out,file=paste0(out,"MARKERS_specific_",k,"_allTIERs.txt"),sep="\t",col.names=T,row.names=F,quote=F)
    print(noquote(paste0('written to file: ', out,"MARKERS_specific_",k,"_allTIERs.txt")))
####.  ----------------------------------   run enrichment
    dat1=final.out[final.out$TIER %in% c("1","2"),]
    g.list=dat1$ID[!duplicated(dat1$ID)]
    invisible(capture.output(enr.result <- enrichr(as.character(g.list),enrich.lists)))
#####.  ---------------------------------    screen overlaps and create list
    for (over in enrich.lists) {
        if(over %in% screen.overlaps){
            print(noquote(paste("           ++++++++ cluster",k, "overlaps to",over,"++++++++     ")))
            print(noquote((enr.result)[[over]][1:5,c(1,2,4)]))
        }
        inter=head((enr.result)[[over]],n=20)
        if (!exists("over.fin")){
            over.fin=inter
        }else {
            over.fin=as.data.frame(rbind(over.fin, inter))
            }
        }
    write.table(over.fin,file=paste0(out,"OVERLAPS_specific_",k,"_TIER_1_2.txt"),sep="\t",col.names=T,row.names=F,quote=F)
    print(noquote(paste0('written to file: ', out,"OVERLAPS_specific_",k,"_TIER_1_2.txt")))
    rm(over.fin)
    
}          

In [ ]:
target=alias2SymbolUsingNCBI("CLEC5A",paste0(w.dir,"Homo_sapiens.gene_info"),required.columns = c("GeneID","Symbol"))
    i=target[1,2]
target
    if (! i %in% rownames(sce.seurat1)){
        print(noquote(paste(marks$Name[k]," serving as Marker for " ,marks$FeatureType[k],"is not in data object and will be skipped!")))
        next
    }else {
       p= FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
        p= p + ggtitle(paste(i,"specific for testted cells?") )
        v=VlnPlot(sce.seurat1, features = as.character(i))    
        print(plot_grid(p, v, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")  )  
    }



In [ ]:
sce.seurat=sce.seurat1
sce.seurat@meta.data$celltypes=rep(NA,nrow(sce.seurat@meta.data))
sce.seurat@meta.data$celltypes[sce.seurat@meta.data$seurat_clusters %in% c("4")]=c("SIGLEC15")
sce.seurat@meta.data$celltypes[sce.seurat@meta.data$seurat_clusters %in% c("2","6")]=c("NKT")
sce.seurat@meta.data$celltypes[sce.seurat@meta.data$seurat_clusters %in% c("0","3","5")]=c("Mono_MACR")
sce.seurat@meta.data$celltypes[sce.seurat@meta.data$seurat_clusters %in% c("1","8","7")]=c("unknown")

table(sce.seurat@meta.data$celltypes)
print(noquote(c("check NAs :")))
table(is.na(sce.seurat@meta.data$celltypes))

sce.seurat@meta.data=sce.seurat@meta.data[,!colnames(sce.seurat@meta.data) %in% c("old.ident","SCT_snn_res.0.3")]
sce.seurat@meta.data$seurat_clusters=paste(sample_ID,sce.seurat@meta.data$seurat_clusters,sep="_")

In [ ]:
dat1=sce.seurat
save(dat1,file=paste0(dir,"QC_ok_",sample_ID,".RData"))

## create PseudoBULK sample

In [ ]:
load(paste0(dir,"QC_ok_",sample_ID,".RData"))

In [ ]:
head(dat1@meta.data)

In [ ]:
table(dat1@meta.data$seurat_clusters)

In [ ]:
inter=dat1@assays$RNA@counts[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_0")]
test=as.data.frame(rowSums(inter))
test$ID=as.character(rownames(test))
colnames(test)[1]=c("MF245_CD45_C0_MONO_MACR")
head(test)
inter=dat1@assays$RNA@counts[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_4")]
test2=as.data.frame(rowSums(inter))
colnames(test2)[1]=c("MF245_CD45_C4_SIGLEC15")
head(test2)
inter=dat1@assays$RNA@counts[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_3")]
test3=as.data.frame(rowSums(inter))
colnames(test3)[1]=c("MF245_CD45_C3_MONO_MACR")
head(test3)
inter=dat1@assays$RNA@counts[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_2","MF245_CD45_6")]
test4=as.data.frame(rowSums(inter))
colnames(test4)[1]=c("MF245_CD45_C6_C2_NKT")
head(test4)
export=as.data.frame(cbind(test,test2,test3,test4))
head(export)

write.table(export,file="~/granuloma_nov2020/pseudo_bulk_sample/MF245_CD45.txt",sep="\t",col.names=T,row.names=F,quote=F)

In [ ]:
inter=dat1@assays$RNA@counts[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_4")]
test=as.data.frame(rowSums(inter))
test$ID=as.character(rownames(test))
colnames(test)[1]=c("MF245_CD45_C4_SIGLEC15")
head(test)

## Prep data for Entropy and Velocity - simple

In [ ]:
load(paste0(dir,"QC_ok_",sample_ID,".RData"))

In [ ]:
head(dat1@meta.data)
table(dat1@meta.data$seurat_clusters,dat1@meta.data$celltypes)

In [ ]:
dim(dat1)
dat2=dat1[,dat1@meta.data$seurat_clusters %in% c("MF245_CD45_0","MF245_CD45_1","MF245_CD45_3","MF245_CD45_4","MF245_CD45_5","MF245_CD45_7")]
dim(dat2)

In [ ]:
dat2=RunPCA(dat2, verbose = TRUE)
Seurat::DimHeatmap(dat2, dims =1 , cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::DimHeatmap(dat2, dims = 2, cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::ElbowPlot(dat2)


In [ ]:
sig_PCs=c(1:13)
dat1 <- RunUMAP(dat2,dims = sig_PCs)
sce.seurat1=Seurat::FindNeighbors(dat2,reduction = "pca",k.param = 20,dims=sig_PCs) 

In [ ]:
i=c("SIGLEC15")  ## MMP9 MKI67 CD4 CD14 SLC9B2 CD68
#MKI67 SLC9B2 SIGLEC15  COL1A1  KRT5  KRT14 PECAM1  CD14 CD4  PTPRC   CD3E   MRC1  NKG7
p=FeaturePlot(sce.seurat1, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
    print(p)
#sce.seurat2=sce.seurat1[,sce.seurat1@reductions$umap@cell.embeddings[,1]> -5 ]
#p=FeaturePlot(sce.seurat2, features = as.character(i),min.cutoff=0.1,max.cutoff=5)
#    print(p)
#sce.seurat1=sce.seurat2
#dat2=sce.seurat2

In [ ]:
save(sce.seurat1,file=paste0(w.dir,"input_data/QC_ok_",sample_ID,"_MONO_MACR_ONLY.RData"))

In [ ]:
write.table(as.data.frame(dat2@assays$RNA@counts),file=paste0(w.dir,"entropie_input/",sample_ID,"for_CytoTRACE.txt"),sep="\t",col.names=T,row.names=T,quote=F)
head(as.data.frame(dat2@assays$RNA@counts))
write.table(as.data.frame(sce.seurat1@reductions$umap@cell.embeddings),file=paste0(w.dir,"entropie_input/",sample_ID,"_UMAP_for_CytoTRACE.txt"),sep="\t",col.names=T,row.names=T,quote=F)
write.table(as.data.frame(colnames(dat2)),file=paste0(w.dir,"entropie_input/",sample_ID,"for_VELOCYTO.txt"),col.names=F,row.names=F,quote=F)